# 画像データの紹介

コンピュータビジョンでは、通常、次のような問題のいずれかを解決します。

* **画像分類**は、最も単純なタスクで、画像をあらかじめ定義された多くのカテゴリーのいずれかに分類する必要があります。例えば、写真上の猫と犬を区別したり、手書きの数字を認識したりします。

* **オブジェクト検出**は、もう少し難しいタスクで、画像上の既知のオブジェクトを見つけて、それらをローカライズする必要があります。

* **領域検出** は、オブジェクト検出に似ていますが、バウンディングボックスを与える代わりに、認識されたオブジェクトのそれぞれの輪郭を示す正確なピクセルマップを返す必要があります。

![ネコ、イヌ、アヒルを使ったコンピュータビジョンによる物体検出のイメージ。](https://i.stack.imgur.com/mFBCV.png)

画像は以下より引用 [CS224d Stanford Course](https://cs224d.stanford.edu/index.html)

## テンソルとしての画像

コンピュータビジョンは画像を扱います。ご存知のように、画像はピクセルで構成されており、ピクセルの長方形の集まり（配列）と考えることができます。

このチュートリアルの最初の部分では、手書きの数字の認識を扱います。ここでは、28x28ピクセルの手書き数字のグレースケール画像で構成されたMNISTデータセットを使用します。各画像は，28x28の配列で表され、この配列の要素は対応するピクセルの強度を表します。0から1の範囲（この場合，浮動小数点数が使われます）か、0から255の範囲（整数）のいずれかです。多次元配列を効率的に扱うことができるため、コンピュータビジョンの分野ではpythonの人気ライブラリである`numpy`がよく使われています。

カラー画像を扱うためには、色を表現する方法が必要です。多くの場合、各画素を赤（R）、緑（G）、青（B）の3つの成分に対応する3つの強度値で表現します。このような色の符号化をRGBといいます。したがって、$W\times H$の大きさのカラー画像は、$3\times H\times W$の大きさの配列で表されます（成分の順番が異なる場合もありますが、考え方は同じです）。

多次元配列は**テンソル**とも呼ばれます。テンソルを使って画像を表現することにも利点があり、一連の画像を格納するのに余分な次元を使うことができる。例えば、800x600次元の200フレームからなるビデオの断片を表現するには、200x3x600x800のサイズのテンソルを使うことができます。

## パッケージのインポートとMNISTデータセットのロード

In [2]:
#Import the packages needed.
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np

PyTorchには[number of datasets](https://pytorch.org/vision/stable/datasets.html)というライブラリが用意されています。ここでは、PyTorchの`torchvison.datasets.MNIST`から利用できる、よく知られた手書きの数字のデータセットである[MNIST](http://yann.lecun.com/exdb/mnist/)を使用しています。データセットオブジェクトはPython Imagine Library (PIL) の画像形式でデータを返します。この画像を`transform=ToTensor()`パラメータを渡してテンソルに変換します．

自分のノートブックを使用する際には、他の内蔵データセット、特に[FashionMNIST](https://pytorch.org/vision/stable/datasets.html#fashion-mnist)で実験することもできます。

In [3]:
from torchvision.transforms import ToTensor

data_train = torchvision.datasets.MNIST('./data', download=True, train=True, transform=ToTensor())
data_test = torchvision.datasets.MNIST('./data', download=True, train=False, transform=ToTensor())

9913344it [03:26, 48052.05it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


29696it [00:00, 192745.77it/s]                          


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


## ディジットデータセットの可視化

データセットをダウンロードしたので、いくつかの数字を可視化してみましょう。

In [ ]:
fig,ax = plt.subplots(1,7)
for i in range(7):
    ax[i].imshow(data_train[i][0].view(28,28))
    ax[i].set_title(data_train[i][1])
    ax[i].axis('off')

## データセットの構造

合計6000枚のトレーニング画像と1000枚のテスト画像があります。トレーニング用とテスト用にデータを分けることが重要です。また、データがどのようなものかを把握するために、データの探索を行いたいと思います。

各サンプルは、以下の構造のタプルです。

* 最初の要素は，1x28x28 サイズのテンソルです。
* 2番目の要素は，テンソルがどの桁を表すかを示す**ラベル**です。

In [ ]:
print('Training samples:', len(data_train))
print('Test samples:', len(data_test))

print('Tensor size:', data_train[0][0].size())
print('First 10 digits are:', [data_train[i][1] for i in range(10)])

画像のすべてのピクセル強度は、0～1の浮動小数点値で表されます。

In [ ]:
print('Min intensity value: ', data_train[0][0].min().item())
print('Max intensity value: ', data_train[0][0].max().item())

独自の画像を読み込む場合は、ニューラルネットワークの学習を開始する前に、すべての値が[**0 - 1**]の範囲にスケーリングされていることを確認することが重要です。

トレーニングデータとテストデータを読み込んで、最初のニューラルネットワークのトレーニングを開始する準備ができました。

## 演習問題

1. 2秒間の320x200のビデオクリップを30フレーム/秒で表現するためのテンソルの大きさはどうなるか？

- 30x200x320
- 60x200x320
- 60x3x200x320
  > 正解
- 2x200x320
